# 3d UNet first experiments

In [ ]:
from tensorflow.keras.layers import Input, Conv3D, Conv3DTranspose, MaxPool3D, UpSampling3D, Dense
from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

import h5py
import numpy as np

In [ ]:
# download data
!wget -O sim_data.h5 'https://data.4tu.nl/file/9f2f96b0-99a6-439b-848e-e914f51d7d85/83b4abe7-54f1-4f36-8e2b-6e2e604adca0'

--2023-06-21 12:35:36--  https://data.4tu.nl/file/9f2f96b0-99a6-439b-848e-e914f51d7d85/83b4abe7-54f1-4f36-8e2b-6e2e604adca0
Resolving data.4tu.nl (data.4tu.nl)... 131.180.169.22
Connecting to data.4tu.nl (data.4tu.nl)|131.180.169.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 38240648 (36M) [application/octet-stream]
Saving to: ‘sim_data.h5’

sim_data.h5         100%[===================>]  36.47M  8.54MB/s    in 5.1s    

2023-06-21 12:35:42 (7.21 MB/s) - ‘sim_data.h5’ saved [38240648/38240648]



## Preprocess data

In [ ]:
with h5py.File('sim_data.h5') as f:
  data = np.array(f['OutArray'])
data.shape

(182, 160, 164)

In [ ]:
# keras expects a feature dimension as the last axis
# here there is only one feature and it doesn't have its own axis, so add it
train_data = tf.expand_dims(data, 3)
print(train_data.shape)
# add a batch dimension
train_data = tf.expand_dims(train_data, 0)
print(train_data.shape)
# not sure if it matters but i always think of the dimensions as being in the order
# width, height, depth, so transpose
train_data = tf.transpose(train_data, (0, 2, 3, 1, 4))
print(train_data.shape)

# just for convenience, trim data
spatial_dim = 128
train_data = train_data[:, :spatial_dim, :spatial_dim, :spatial_dim, :]
print(train_data.shape)

(182, 160, 164, 1)
(1, 182, 160, 164, 1)
(1, 160, 164, 182, 1)
(1, 128, 128, 128, 1)


## Create very simple 3D Unet segmentation model

In [ ]:
num_classes = 3

input = Input(shape=train_data.shape[1:])
x = input

for filters in [2, 4, 8, 16, 32, 64]:
  x = Conv3D(filters=filters, kernel_size=3, padding='same')(x)
  x = MaxPool3D(pool_size=2)(x)

for filters in [64, 32, 16, 8, 4, 2]:
  x = Conv3DTranspose(filters=filters, kernel_size=3, padding='same')(x)
  x = UpSampling3D(2)(x)


output = Dense(units=num_classes, activation='softmax')(x)

model = Model(input, output)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 128, 128, 128, 1  0         
                             )]                                  
                                                                 
 conv3d (Conv3D)             (None, 128, 128, 128, 2)  56        
                                                                 
 max_pooling3d (MaxPooling3D  (None, 64, 64, 64, 2)    0         
 )                                                               
                                                                 
 conv3d_1 (Conv3D)           (None, 64, 64, 64, 4)     220       
                                                                 
 max_pooling3d_1 (MaxPooling  (None, 32, 32, 32, 4)    0         
 3D)                                                             
                                                             

## Apply it to data

In [ ]:
model_output = model(train_data)
model_output.shape

TensorShape([1, 128, 128, 128, 3])

## Create fake labels

In [ ]:
# set label to 0 if negative, 1 is positive, or 2 if > 0.1
train_x = train_data
zeros = tf.zeros_like(train_data)
ones = tf.ones_like(train_data)
twos = 2 * ones
train_y = tf.where(train_data > 0, ones, zeros)
train_y = tf.where(train_data > 0.1, twos, train_y)
train_y.shape

TensorShape([1, 128, 128, 128, 1])

## Compile model

In [ ]:
optimizer = Adam()

model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy')

## Train the model

In [ ]:
model.fit(train_x, train_y, epochs=100)

Epoch 1/100
1/1 [==============================] - 10s 10s/step - loss: 1.1021
Epoch 2/100
1/1 [==============================] - 0s 58ms/step - loss: 1.0628
Epoch 3/100
1/1 [==============================] - 0s 57ms/step - loss: 0.9197
Epoch 4/100
1/1 [==============================] - 0s 57ms/step - loss: 1.0365
Epoch 5/100
1/1 [==============================] - 0s 57ms/step - loss: 0.8343
Epoch 6/100
1/1 [==============================] - 0s 55ms/step - loss: 0.8725
Epoch 7/100
1/1 [==============================] - 0s 56ms/step - loss: 0.8952
Epoch 8/100
1/1 [==============================] - 0s 57ms/step - loss: 0.8740
Epoch 9/100
1/1 [==============================] - 0s 56ms/step - loss: 0.8236
Epoch 10/100
1/1 [==============================] - 0s 58ms/step - loss: 0.7846
Epoch 11/100
1/1 [==============================] - 0s 65ms/step - loss: 0.8189
Epoch 12/100
1/1 [==============================] - 0s 55ms/step - loss: 0.8202
Epoch 13/100
1/1 [==============================]